In [1]:
import pandas as pd
import numpy as np
import datetime, re, warnings
from fuzzywuzzy import fuzz, process
from pandarallel import pandarallel

In [2]:
#Run notebook in conda env lexnlp
import lexnlp.extract.en.definitions
import lexnlp.extract.en.amounts
import lexnlp.extract.en.regulations

/Users/katherinewood/anaconda3/envs/lexnlp/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('expand_frame_repr', False)

In [3]:
reports = pd.read_csv('SPDB/IncidentReports.csv', encoding="ISO-8859-1", dtype='object')
recalls = pd.read_csv('recalls.csv', 
                      dtype={'RecallID':'Int64', 'RecallNumber': str, 'RecallDate': str,
                             'Description': str, 'URL': str, 'Title': str, 'ConsumerContact': str,
                             'LastPublishDate': str, 'Images': 'object', 'SoldAtLabel': str,
                             'Distributors_CompanyID': 'Int64', 'DistributorsSName': str, 
                             'Hazards_HazardType': str, 'Hazards_HazardTypeID': str, 'Hazards_Name': str,
                             'Importers_CompanyID': 'Int64', 'Importers_Name': str, 'Inconjunctions_URL': str,
                             'Injuries_Name': str, 'ManufacturerCountries_Country': str, 
                             'Manufacturers_CompanyID': 'Int64', 'Manufacturers_Name': str, 'ProductUPCs_UPC': str,
                             'Products_CategoryID': 'Int64', 'Products_Description': str, 'Products_Model': str,
                             'Products_Name': str, 'Products_NumberOfUnits': str, 'Products_Type': str,
                             'Remedies_Name': str, 'RemedyOptions_Option': str, 'Retailers_CompanyID': str,
                             'Retailers_Name': str})

In [609]:
#Munge and infill whatever information we can extract from the recall descriptions

##UPCs loaded as strings; clean for whitespace & non-numeric chars
recalls['ProductUPCs_UPC'] = recalls['ProductUPCs_UPC'].str.replace(' |-|\.', '')
recalls = recalls.rename(columns={'ProductUPCs_UPC': 'UPC'})
#Extract unit numbers from string phrases (e.g. "About 35")
num_units = recalls['Products_NumberOfUnits'].str.replace(',', '')
num_units = num_units.str.extract(r'(\d+)', expand=False).astype('float')
recalls['Products_NumberOfUnits'] = num_units
#Extract total number of complaints from the string column
#Parse dates from strings
recalls['RecallDate'] = pd.to_datetime(recalls['RecallDate'])
recalls['LastPublishDate'] = pd.to_datetime(recalls['LastPublishDate'])
#Break the standardized titles into helpful fields; standardized
#titles take the form "[Company] recalls [product] due to [hazard]"
titles = recalls['Title'].str.split('[Re]?[A]?[a]?[nnounce]?[s]?Recall[s]?[ed]?|Due to', expand=True)
titles = titles.rename(columns={0: 'CompanyShortname', 1: 'ProductsShortname', 2: 'HazardAlt'})
recalls = pd.concat([recalls, titles], axis=1)
recalls = recalls[recalls['RecallDate'] > pd.to_datetime('01/01/2000')]

In [68]:
#Vectorize over a series of strings
# TODO can we recognize ranges of serial numbers ("between NF830 and NF 960")
# TODO can we de-stem serial numbers with x-fillers? (NF687xxxx)
def extract_probable_specifiers(text):
    if pd.isnull(text):
        text=''
    pattern = r"(([0-9A-Z])+[a-z]*([\\-]?[\\.*]?[0-9A-Z]*)*){2,}"
    matches = re.finditer(pattern, text)
    unique_matches = set([match.group() for matchNum, match in enumerate(matches)])
    return list(unique_matches)

In [546]:
#reports = pd.concat([v_reports, b_reports], axis=0, sort=False)
#recalls = pd.concat([v_recalls, b_recalls], axis=0, sort=False)

#Funnel match
#Preparation: extract possible brands from the fields likely to contain them, for both the reports and the recalls.
#Brands may not be referred to by a consistent name across the two datasets.
def prepare_fields(reports, recalls):
    brand_from_comments = [report[1] if not isinstance(report, float) else '' for report in reports['Company Comments'].str.split('\\) |:')]
    reports['candidate_brand'] = list(zip(reports['Brand'], 
                                               reports['Manufacturer / Importer / Private Labeler Name'],
                                               brand_from_comments))
    reports['candidate_brand'] = reports['candidate_brand'].apply(lambda x: list(set(x)))
    recalls['candidate_brand'] = list(zip(recalls['CompanyShortname'], recalls['Manufacturers_Name'],
                                         recalls['Distributors_Name'], recalls['Importers_Name']))
    recalls['candidate_brand'] = recalls['candidate_brand'].apply(lambda x: list(set(x)))

    #Preparation: extract alphanumeric strings that are likely to be model numbers, serial numbers, or UPCs.
    #Recall notices are very unlikely to have the dedicated fields populated, but tend to mention them in the
    #text of the recall announcement.
    #Reports tend to have this information in the dedicated fields, but as a precaution we also try to pull it
    #from the unstructured text.

    reports['specifiers'] = [extract_probable_specifiers(report) for report in 
                             [reports['Product Description'] + ' '+ reports['Incident Description']][0]]

    recalls['specifiers'] = [extract_probable_specifiers(recall) for recall in recalls['Description']]
    
    return reports, recalls

#Phase 1
#For each complaint:
#for each candidate brand: 'CompanyShortname', 'Manufacturers_Name', 'Distributors_Name', 'Importers_Name'
# - Fuzzy match to the possible brands list from each recall notice
# - If any matches score > threshold, save recall as candidate (add a column that contains a list of probable recall IDs)
# - If no matches score > threshold, label complaint as "no recall"
#Phase 2
#For all complaints that have candidate recall numbers:
#For each candidate recall:
# - Fuzzy match to possible products using product type from report & product name from recall
# - Retain match if score > threshold on any
#Finally, take the specifiers extracted from the recall description and look for an exact match in the following order:
# - Model name or number
# - Serial number
# - UPC
# - Specifiers extracted from the product description

# TODO badly need to optimize; convert to map() instead of nested fors; collapse brands together,
# https://github.com/nalepae/pandarallel still too slow
# consider looking for the phase "not recalled" or "no recall" in any of the text
# add consensus measures
# todo the associated report numbers do exist, in a few cases

def find_match(report, recalls, threshold=60):
    match_ids = []
    for c in report['candidate_brand']:
        if pd.isnull(c) or not c:
            continue
        else:
            match_ids += [recalls.iloc[r]['RecallID'] for r in range(len(recalls)) if
                        (pd.Series(list(zip(*process.extract(c, 
                                                             recalls.iloc[r]['candidate_brand'], 
                                                             limit=len(recalls.iloc[r]['candidate_brand']),
                                                            scorer = fuzz.token_set_ratio)))[1]) > threshold).any()]
    if not match_ids:
        return 0
    else:
        candidate_recalls = recalls[recalls['RecallID'].apply(lambda x: x in match_ids)]
        product_match = process.extract(report['Product Type'], list(candidate_recalls['Products_Name']), 
                                        limit=len(candidate_recalls), scorer = fuzz.token_set_ratio)
        v = pd.Series(list(zip(*product_match))[1]).values
        candidate_recalls = candidate_recalls[pd.Series(list(zip(*product_match))[1]).values > threshold]
        if candidate_recalls.empty:
            return 0
        else:
            reported_specs = [report[spec].lower() 
                              if not pd.isnull(report[spec]) else '' 
                              for spec in ['Model Name or Number', 'Serial Number', 'UPC']]
                        
            reported_spec_matches = candidate_recalls['specifiers'].apply(
                lambda x: any([spec.lower() in report_spec.lower() for spec in x for report_spec in reported_specs]))
            
            definite_match = candidate_recalls['RecallID'][reported_spec_matches]
                        
            if len(definite_match) > 1:
                warnings.warn('More than one "unique" match found')
                return definite_match.min()
            if len(definite_match) > 0:
                return definite_match.values[0]
            else:
                bag_specifier_matches = candidate_recalls['specifiers'].apply(
                lambda x: any([spec.lower() in report_spec.lower() 
                               for spec in x 
                               for report_spec in report['specifiers']]))
                bag_matches = candidate_recalls['RecallID'][bag_specifier_matches]
                if len(bag_matches) > 1:
                    warnings.warn('More than one "unique" match found')
                    return bag_matches.min()
                return bag_matches.values[0] if bag_specifier_matches.sum() else 0

In [497]:
def get_matched_brands(brand, reports, recalls):
    brand_reports = reports[reports['Manufacturer / Importer / Private Labeler Name'].str.contains(brand, case=False) 
                            | reports['Brand'].str.contains(brand, case=False)
                            | reports['Incident Description'].str.contains(brand, case=False)]
    brand_recalls = recalls[recalls['CompanyShortname'].str.contains(brand, case=False) 
                            | recalls['Manufacturers_Name'].str.contains(brand, case=False) 
                            | recalls['Importers_Name'].str.contains(brand, case=False) 
                            | recalls['Distributors_Name'].str.contains(brand, case=False) 
                            | recalls['Retailers_Name'].str.contains(brand, case=False)]
    return (brand_reports, brand_recalls)


In [574]:
reports, recalls = prepare_fields(reports, recalls)

In [576]:
reports['labels'] = reports.parallel_apply(lambda row: find_match(row, recalls, threshold=80), axis=1)

KeyboardInterrupt: 